In [ ]:
# Step 1 - Scraping

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import time
from splinter import Browser

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'


browser.visit(url)
time.sleep(10)

html = browser.html
soup = bs(html, 'html.parser')



In [4]:

results = soup.find_all('div', class_='content_title')


In [5]:
len(results)

53

In [6]:
news_title = results[1].text

In [7]:
news_title

'Independent Review Indicates NASA Prepared for Mars Sample Return Campaign'

In [8]:
results = soup.find_all('div', class_='article_teaser_body')

In [9]:
news_p = results[0].text

In [10]:
news_p

'NASA released an independent review report Tuesday indicating the agency is well positioned for its Mars Sample Return campaign to bring pristine samples from Mars to Earth for scientific study.'

In [10]:
results = soup.find_all('div', class_='list_date')

In [14]:
news_date = results[0].text

In [15]:
results = soup.find_all('div',class_='list_image')

In [21]:
news_img_relative_url = results[0].find('img')['src']
base_url = "https://mars.nasa.gov/"
news_img_complete_url = base_url+news_img_relative_url

In [22]:
news_img_complete_url

'https://mars.nasa.gov//system/news_items/list_view_images/8794_PIA23496-320.jpg'

In [11]:
# JPL Mars Space Images - Featured Image

In [12]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [13]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [14]:
html = browser.html
soup = bs(html, 'html.parser')

In [15]:
relative_featured_image_url = soup.find_all('article', class_='carousel_item')[0]['style'].split("url('")[1][:-3]


In [16]:
base_url = "https://www.jpl.nasa.gov"
featured_image_url = base_url + relative_featured_image_url

In [17]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18429-1920x1200.jpg'

In [18]:
# Mars Facts

In [19]:
import pandas as pd

In [20]:
url = "https://space-facts.com/mars/"
tables = pd.read_html(url)
df = tables[0]
# df.columns=['Equatorial_Diameter','Polar_Diameter','Mass','Moons','Orbit_Distance','Orbit_Period','Surface_Temperature','First_Record','Recorded_By']

In [21]:
df = df.rename(columns={0:"",1:"Mars"}).set_index("")

In [22]:
df

,Mars
,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [23]:
df.to_html('table.html',border=0,justify="left",classes="table table-striped",)

In [56]:
# Mars Hemispheres

In [24]:
url_list = ["https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced",
"https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced",
"https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced",
"https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"]



In [128]:
import threading

In [ ]:
def create_dic():
    

In [130]:
url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
t1 = threading.Thread(target=requests.get(url))
t2 = threading.Thread(target=bs(response.text,'html.parser'))

t1.start()
t2.start()

t1.join()
t2.join()


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/Users/Sean/opt/anaconda3/envs/PythonData/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/Sean/opt/anaconda3/envs/PythonData/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'Response' object is not callable



In [26]:
hemisphere_image_urls=[]
for url in url_list:
    response=requests.get(url)
    soup=bs(response.text,'html.parser')
    
    
    url_base = "https://astrogeology.usgs.gov/"

    relative_path = soup.find_all('img', class_='wide-image')[0]['src']
    img_url = url_base + relative_path

    title = soup.find_all('h2', class_='title')[0].text.split(' Hemisphere')[0]

    hemisphere_image_urls.append({"title":title,"img_url":img_url})


In [27]:
hemisphere_image_urls

[{'title': 'Cerberus',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [ ]:
# Step 2 - MongoDB and Flask Application

In [29]:
hemisphere_image_urls[0]['img_url']

'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'